# pangeo ERA data

In [ ]:
#notebook based on zflemings:https://nbviewer.jupyter.org/github/zflamig/dask-era5/blob/main/notebook/era5_fargate_dask.ipynb
import matplotlib.pyplot as plt
import xarray as xr
import fsspec
import dask
import s3fs
import numpy as np
import gcsfs
import fsspec
import xarray as xr

xr.set_options(display_style="html")  #display dataset nicely 

In [ ]:

with xr.open_zarr(fsspec.get_mapper('gcs://pangeo-era5/reanalysis/spatial-analysis'),
                  consolidated=True,
                  chunks='auto') as ds:
    print(ds)

In [ ]:

from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")
ds  = cat["era5_hourly_reanalysis_single_levels_sa"].to_dask()
ds_wsp = np.sqrt(ds.u10**2 + ds.v10**2)

In [ ]:
ds_wsp

In [ ]:
#ds_wsp.sel(time='2017-10-08T00',longitude=slice(115,240),latitude=slice(50,25)).plot(vmin=0,vmax=20)
ds_wsp.sel(time='2017-10-08T00',longitude=slice(115,240),latitude=slice(50,25)).plot(vmin=0,vmax=20)

In [ ]:
# Visualizations
import hvplot.xarray
import hvplot.pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cartopy.crs as crs

ds2 = ds_wsp.sel(time=slice('2017-09-15T00','2017-10-08T00'),longitude=slice(115,240),latitude=slice(60,25))
ds2 = ds2.resample(time='1D').mean()
proj = crs.Orthographic(-180, 30)
ds2.hvplot.quadmesh(
    'longitude', 'latitude', projection=proj, project=True,
    cmap='gnuplot', dynamic=True, coastline='50m', 
    frame_width=300, logz=True, clim=(0.01,20), rasterize=True)

In [ ]:
# high winds were evening 10/8/2017 9:45 PM 
ds

In [ ]:
#ds.u10.sel(longitude=slice(122,124),latitude=slice(39,37)).sel(time=slice('2017-10-01T00','2017-10-14')).mean({'latitude','longitude'}).plot()
ds.u10.sel(longitude=122.7,latitude=38.4,method='nearest').sel(time=slice('2017-10-09T00','2017-10-09')).plot()
#ds.v10.sel(longitude=122.7,latitude=38.4,method='nearest').sel(time=slice('2017-10-09T00','2017-10-09')).plot()


In [ ]:
ts = ds_wnd.sel(longitude=slice(236,239),latitude=slice(46,41)).mean({'latitude','longitude'}).load()

In [ ]:
ts

In [ ]:
ts.to_netcdf('../../data/era5ts.nc')

In [ ]:
sub=ts.sel(time=slice('2017','2020'))